In [26]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
import random
import pickle
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score
import seaborn as sns
# from ucimlrepo import fetch_ucirepo 

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split

from data.data_loader import EPCDataset, PowerWeatherDatasetWithSeason, PowerWeatherDataset, SingleTermDataset, MultiTermDataset
from models.lstm import LSTMModel
from models.lstm_attention import LSTMWithAttention, BiLSTMWithAttention
from models.gru import GRUModel
from models.utils import create_model, train_for_short_term_forecast, train_for_long_term_forecast, load_model, evaluate_for_short_term_forecast, evaluate_for_long_term_forecast

from explainers.utils import get_explainer
# from explainers.lime import LimeExplainer
# from explainers.shap import ShapExplainer
# from explainers.attention import AttentionExplainer
# from explainers.grad_cam import GradCAMExplainer
# from explainers.lrp import LRPExplainer

In [27]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [28]:
# short term 
features_for_1h = [
    'Global_active_power',
    'Global_intensity',
    'Sub_metering_1', 
    'Sub_metering_2', 
    'Sub_metering_3', 
    'Temperature',	
    'Humidity',	
]

# long term
features_for_6h = [
    'Global_active_power',
    'Global_intensity',
    'Sub_metering_1', 
    'Sub_metering_2', 
    'Sub_metering_3', 
    'Temp_Min',	'Temp_Max', 'Temp_Avg',	'Temp_Range',
    'Humidity_Min',	'Humidity_Max',	'Humidity_Avg',	'Humidity_Range'
]

file_path_for_1h = 'data/final_data.csv'
file_path_for_6h = 'data/final_data_per_6hr_with_avg_range.csv'

In [29]:
def load_dataset(params, target_features_long, target_features_short, is_long_term_forecast=True):
    selected_features = dict()
    
    if is_long_term_forecast:
        dataset_6h = EPCDataset(
            # file_path=file_path_for_6h,
            file_path=file_path_for_1h,
            sequence_length=params['long_term_length'],  
            prediction_length=params['long_term_pred_length'],
            target_features=target_features_long
        )
        train_long, train_targets_long, eval_long, eval_targets_long = dataset_6h.load_data()
        
        dataset_1h = EPCDataset(
            file_path=file_path_for_1h,
            sequence_length=params['short_term_length'],  
            prediction_length=params['short_term_pred_length'],
            target_features=target_features_short
        )
        train_short, train_targets_short, eval_short, eval_targets_short = dataset_1h.load_data()
        
        train_data=(train_long, train_short)
        train_targets=(train_targets_long, train_targets_short)
        eval_data=(eval_long, eval_short)
        eval_targets=(eval_targets_long, eval_targets_short)

        selected_features['long'] = dataset_6h.selected_features
        selected_features['short'] = dataset_1h.selected_features

    else:
        dataset_1h = EPCDataset(
            file_path=file_path_for_1h,
            sequence_length=params['sequence_length'],  
            prediction_length=params['prediction_length'],
            target_features=target_features_short
        )
        
        train_sequence, train_targets_sequence, eval_sequence, eval_targets_sequence = dataset_1h.load_data()

        train_data=train_sequence
        train_targets=train_targets_sequence
        eval_data=eval_sequence
        eval_targets=eval_targets_sequence
    
        selected_features['single'] = dataset_1h.selected_features

    return train_data, train_targets, eval_data, eval_targets, selected_features

In [30]:
# Task
## long-term forecast: 일주일 전력량 예측 (7*24) -> Long-term Short-term CNN-LSTM 모델
## Short-term forecast: 하루 전력량 예측 (24)    -> LSTM, GRU, CNN-LSTM 모델 

# 1. 온/습도 정보 없는 케이스도 비교 필요 


dataset_params = {
    # for long-term forecast 

    # 'long_term_length' : 365*4, # 6시간 단위 1년 데이터
    'long_term_length' : 90*24,
    'long_term_pred_length' : 256, # 64, 128, 256
    
    'short_term_length' : 30*24, # 1시간 단위 한달 데이터
    'short_term_pred_length' : 7*24, # 일주일 데이터 예측

    
    # 2. LSTM, GRU, CNN-LSTM  -> 365*24 / 7*24 (비교용)
    # for Short-term forecast 
    'sequence_length' : 24*30,  # -> 일주일 / 10일 / 15일 / 한달  
    'prediction_length' : 24
}

In [31]:
is_long_term_forecast = True

train_data, train_targets, eval_data, eval_targets, selected_features = load_dataset(params=dataset_params,
                                                                                     target_features_long=features_for_1h, 
                                                                                     target_features_short=features_for_1h, 
                                                                                     is_long_term_forecast=True)
if isinstance(train_data, tuple):
    print("6-hour data sequences for long-term forecast:")
    print(f"  Train sequences shape: {train_data[0].shape}")
    print(f"  Train targets shape: {train_targets[0].shape}")
    print(f"  Eval sequences shape: {eval_data[0].shape}")
    print(f"  Eval targets shape: {eval_targets[0].shape}")
    print(f"  Selected_features: {selected_features['long']}")
    
    print("\n1-hour data sequences for long-term forecast:")
    print(f"  Train sequences shape: {train_data[1].shape}")
    print(f"  Train targets shape: {train_targets[1].shape}")
    print(f"  Eval sequences shape: {eval_data[1].shape}")
    print(f"  Eval targets shape: {eval_targets[1].shape}")
    print(f"  Selected_features: {selected_features['short']}")
    
else:
    print("1-hour data sequences for short-term forecast:")
    print(f"  Train sequences shape: {train_data.shape}")
    print(f"  Train targets shape: {train_targets.shape}")
    print(f"  Eval sequences shape: {eval_data.shape}")
    print(f"  Eval targets shape: {eval_targets.shape}")
    print(f"  Selected_features: {selected_features['single']}")

6-hour data sequences for long-term forecast:
  Train sequences shape: torch.Size([22588, 2160, 13])
  Train targets shape: torch.Size([22588, 256])
  Eval sequences shape: torch.Size([5647, 2160, 13])
  Eval targets shape: torch.Size([5647, 256])
  Selected_features: ['Global_active_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Temperature', 'Humidity', 'sin_hour', 'cos_hour', 'sin_day', 'cos_day', 'sin_month', 'cos_month']

1-hour data sequences for long-term forecast:
  Train sequences shape: torch.Size([23810, 720, 13])
  Train targets shape: torch.Size([23810, 168])
  Eval sequences shape: torch.Size([5953, 720, 13])
  Eval targets shape: torch.Size([5953, 168])
  Selected_features: ['Global_active_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Temperature', 'Humidity', 'sin_hour', 'cos_hour', 'sin_day', 'cos_day', 'sin_month', 'cos_month']


In [32]:
def load_trained_model(params):
    if params is None:
        return

    # Extract parameters
    model_name = params['model_name']
    hidden_size = params['hidden_size']
    num_layers = params['num_layers']
    dropout = params['dropout']
    num_epochs = params['num_epochs']
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    patience = params['patience']
    mse_decay = params['mse_decay']

    # Determine input size and output size
    input_size = {
        # 'long': len(features_for_6h)+6,
        'long': len(features_for_1h)+6,
        'short': len(features_for_1h)+6,
        'single': len(features_for_1h)+6
    }
    output_size = {
        'long': dataset_params['long_term_pred_length'],
        'short': dataset_params['short_term_pred_length'],
        'single': dataset_params['prediction_length']
    }

    if mse_decay:
        mse_alpha = 0.3
        mse_beta = 1.0

    else:
        mse_alpha = 1.0
        mse_beta = 1.0

    # Model 생성 
    model = create_model(
        model_name=model_name,
        input_size=input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        output_size=output_size,
        dropout=dropout, 
        long_term_length=dataset_params['long_term_length'], 
        short_term_length=dataset_params['short_term_length']
    ) 

    if 'LS_CNNLSTM' in model_name:
        model_path = './trained_models/{}_long_{}_short_{}_{}_{}_{}_alpha_{}_beta_{}.pth'.format(
            model_name, dataset_params['long_term_length'], dataset_params['short_term_length'],
            dataset_params['long_term_pred_length'], hidden_size, num_layers, mse_alpha, mse_beta
        )
    else:
        model_path = './trained_models/{}_{}_{}.pth'.format(
            model_name, dataset_params['sequence_length'], dataset_params['prediction_length']
        )
        
    print("Model path:", model_path)

    # Load pre-trained model or train a new one
    if os.path.exists(model_path):
        print(f"Loading the pre-trained {model_name} model...")
        model.load_state_dict(torch.load(model_path))
        model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    else:
        print(f"{model_name} model not found. Training a new model...")
        if 'LS_CNNLSTM' in model_name:
            train_for_long_term_forecast(
                model=model,
                model_name=model_name,
                train_data=train_data,
                train_targets=train_targets,
                eval_data=eval_data,
                eval_targets=eval_targets,
                model_path=model_path,
                num_epochs=num_epochs,
                batch_size=batch_size,
                learning_rate=learning_rate,
                patience=patience,
                oversample_eval=True, 
                alpha=mse_alpha, 
                beta=mse_beta
            )
        else:
            train_for_short_term_forecast(
                model=model,
                model_name=model_name,
                train_sequences=train_data,
                train_targets=train_targets,
                eval_sequences=eval_data,
                eval_targets=eval_targets,
                model_path=model_path,
                num_epochs=num_epochs,
                batch_size=batch_size,
                learning_rate=learning_rate,
                patience=patience
            )

    return model, model_path

In [33]:
from itertools import product

In [34]:
model_names = ['LS_CNNLSTM', 'LS_CNNLSTM_Att']
hidden_sizes = [256, 512]
mse_decayes = [True, False]

# results_for_compare = {}
original_results = {} 

# itertools.product를 사용하여 모든 조합 생성
for model_name, hidden_size, mse_decay in product(model_names, hidden_sizes, mse_decayes):
    # print(f"Model: {model_name}, Hidden Size: {hidden_size}, MSE Decay: {mse_decay}")
   
    model_params = {
        'model_name' : model_name, 
        'hidden_size' : hidden_size,
        'num_layers' : 3, 
        'dropout' : 0.3,
        'num_epochs' : 200,
        'batch_size' : 16,
        'learning_rate' : 0.001,
        'patience' : 12,
        'mse_decay' : mse_decay
    }

    model, model_path = load_trained_model(model_params)

    
    print("Evaluating the model...")
    if is_long_term_forecast:
        results = evaluate_for_long_term_forecast(
            model=model,
            eval_data=eval_data,
            eval_targets=eval_targets,
            model_name=model_params['model_name'],
            batch_size=model_params['batch_size'], 
            oversample_eval=True 
        )
    else:
        results = evaluate_for_short_term_forecast(
            model=model,
            eval_sequences=eval_short,
            eval_targets=eval_targets_short,
            model_name=model_params['model_name'],
            batch_size=model_params['batch_size']
        )

    key = '{}_{}_{}'.format(model_name, hidden_size, mse_decay)
    original_results[key] = results

Model path: ./trained_models/LS_CNNLSTM_long_2160_short_720_256_256_3_alpha_0.3_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM model...
Evaluating the model...


/tmp/ipykernel_20448/3867406579.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


R² Score: 0.8597
Adjusted R²: 0.8594
SMAPE: 33.18
MASE: 0.5526
Model path: ./trained_models/LS_CNNLSTM_long_2160_short_720_256_256_3_alpha_1.0_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM model...
Evaluating the model...


/tmp/ipykernel_20448/3867406579.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


R² Score: 0.8486
Adjusted R²: 0.8483
SMAPE: 33.69
MASE: 0.5715
Model path: ./trained_models/LS_CNNLSTM_long_2160_short_720_256_512_3_alpha_0.3_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM model...


...

Evaluating the model...
R² Score: 0.7085
Adjusted R²: 0.7079
SMAPE: 41.08
MASE: 0.7915
Model path: ./trained_models/LS_CNNLSTM_Att_long_2160_short_720_256_512_3_alpha_0.3_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM_Att model...


/tmp/ipykernel_20448/3867406579.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Evaluating the model...
R² Score: 0.7782
Adjusted R²: 0.7777
SMAPE: 38.92
MASE: 0.6968
Model path: ./trained_models/LS_CNNLSTM_Att_long_2160_short_720_256_512_3_alpha_1.0_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM_Att model...


/tmp/ipykernel_20448/3867406579.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Evaluating the model...
R² Score: 0.8145
Adjusted R²: 0.8141
SMAPE: 37.51
MASE: 0.6376


In [35]:
original_results

{'LS_CNNLSTM_256_True': {'R2 Short': 0.8596751093864441,
  'Adjusted R2': 0.8593679493295362,
  'SMAPE Short': np.float32(33.17787),
  'MASE Short': np.float32(0.55263716)},
 'LS_CNNLSTM_256_False': {'R2 Short': 0.8486469984054565,
  'Adjusted R2': 0.8483156986882097,
  'SMAPE Short': np.float32(33.692307),
  'MASE Short': np.float32(0.57151854)},
 'LS_CNNLSTM_512_True': {'R2 Short': 0.8329382538795471,
  'Adjusted R2': 0.8325725689663351,
  'SMAPE Short': np.float32(33.991646),
  'MASE Short': np.float32(0.5963537)},
 'LS_CNNLSTM_512_False': {'R2 Short': 0.8353075981140137,
  'Adjusted R2': 0.8349470995074271,
  'SMAPE Short': np.float32(34.891926),
  'MASE Short': np.float32(0.59581584)},
 'LS_CNNLSTM_Att_256_True': {'R2 Short': 0.7264457941055298,
  'Adjusted R2': 0.7258470056433934,
  'SMAPE Short': np.float32(39.98949),
  'MASE Short': np.float32(0.7689069)},
 'LS_CNNLSTM_Att_256_False': {'R2 Short': 0.7085404396057129,
  'Adjusted R2': 0.7079024577425834,
  'SMAPE Short': np.floa

In [17]:
import re
import json

# 파일 경로
file_path = "important_features_top5.txt"

# 저장할 딕셔너리
important_features_dict = {}

# 정규 표현식 패턴
model_path_pattern = re.compile(r"\./trained_models/(.+)")
feature_pattern = re.compile(r"([\w_]+): ([\d\.]+)")

features = [
    'Global_active_power',
    'Global_intensity',
    'Sub_metering_1', 
    'Sub_metering_2', 
    'Sub_metering_3', 
    'Temperature',    
    'Humidity',    
]


# 변수 초기화
current_model = None
current_section = None

# 파일 읽기
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()
        
        # 모델 경로 감지 및 모델 키 추출
        match = model_path_pattern.match(line)
        if match:
            current_model = match.group(1)
            important_features_dict[current_model] = {"Important Long-term Features": {}, "Important Short-term Features": {}}
            continue

        # 섹션 감지
        if "Important Long-term Features" in line:
            current_section = "Important Long-term Features"
            continue
        elif "Important Short-term Features" in line:
            current_section = "Important Short-term Features"
            continue
        
        # Feature 값 추출
        match = feature_pattern.match(line)
        if match and current_model and current_section:
            feature_name, score = match.groups()
            if feature_name in features:
                important_features_dict[current_model][current_section][feature_name] = float(score)

# important_features_dict


# for key, important_features in important_features_dict.items():

#     # # print(important_features)
#     # model_name = key.split('_long')[0]
#     # hidden_size = 512 if '512' in key else 256
#     # mse_decay = True if '0.3' in key else False

#     important_features_for_long = []
#     important_features_for_short = []
    
#     for feature_long in important_features['Important Long-term Features']:
#         important_features_for_long.append(feature_long)
    
#     for feature_short in important_features['Important Short-term Features']:
#         important_features_for_short.append(feature_short)

In [16]:
important_features_for_long

['Global_active_power',
 'Global_intensity',
 'Sub_metering_2',
 'Sub_metering_3',
 'Humidity']

In [18]:
def load_dataset_new_features(params, is_long_term_forecast=True):
    selected_features = dict()
    
    if is_long_term_forecast:
        dataset_6h = EPCDataset(
            file_path=file_path_for_1h,
            sequence_length=params['long_term_length'],  
            prediction_length=params['long_term_pred_length'],
            target_features=important_features_for_long
        )
        train_long, train_targets_long, eval_long, eval_targets_long = dataset_6h.load_data()
        
        dataset_1h = EPCDataset(
            file_path=file_path_for_1h,
            sequence_length=params['short_term_length'],  
            prediction_length=params['short_term_pred_length'],
            target_features=important_features_for_short
        )
        train_short, train_targets_short, eval_short, eval_targets_short = dataset_1h.load_data()
        
        train_data=(train_long, train_short)
        train_targets=(train_targets_long, train_targets_short)
        eval_data=(eval_long, eval_short)
        eval_targets=(eval_targets_long, eval_targets_short)

        selected_features['long'] = dataset_6h.selected_features
        selected_features['short'] = dataset_1h.selected_features

    else:
        dataset_1h = EPCDataset(
            file_path=file_path_for_1h,
            sequence_length=params['sequence_length'],  
            prediction_length=params['prediction_length'],
            target_features=important_features_for_1h
        )
        
        train_sequence, train_targets_sequence, eval_sequence, eval_targets_sequence = dataset_1h.load_data()

        train_data=train_sequence
        train_targets=train_targets_sequence
        eval_data=eval_sequence
        eval_targets=eval_targets_sequence
    
        selected_features['single'] = dataset_1h.selected_features

    return train_data, train_targets, eval_data, eval_targets, selected_features

In [19]:
is_long_term_forecast = True

In [20]:
def load_trained_model(params):
    if params is None:
        return

    # Extract parameters
    model_name = params['model_name']
    hidden_size = params['hidden_size']
    num_layers = params['num_layers']
    dropout = params['dropout']
    num_epochs = params['num_epochs']
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    patience = params['patience']
    mse_decay = params['mse_decay']

    # Determine input size and output size
    input_size = {
        # 'long': len(important_features_for_6h)+6,
        'long': len(important_features_for_long)+6,
        'short': len(important_features_for_short)+6,
        'single': len(important_features_for_short)+6
    }
    output_size = {
        'long': dataset_params['long_term_pred_length'],
        'short': dataset_params['short_term_pred_length'],
        'single': dataset_params['prediction_length']
    }

    if mse_decay:
        mse_alpha = 0.3
        mse_beta = 1.0

    else:
        mse_alpha = 1.0
        mse_beta = 1.0

    # Model 생성 
    model = create_model(
        model_name=model_name,
        input_size=input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        output_size=output_size,
        dropout=dropout, 
        long_term_length=dataset_params['long_term_length'], 
        short_term_length=dataset_params['short_term_length']
    ) 

    if 'LS_CNNLSTM' in model_name:
        model_path = './trained_models/important_features_{}_{}_{}_long_{}_short_{}_{}_{}_{}_alpha_{}_beta_{}.pth'.format(
            input_size['long'], input_size['short'],
            model_name, dataset_params['long_term_length'], dataset_params['short_term_length'],
            dataset_params['long_term_pred_length'], hidden_size, num_layers, mse_alpha, mse_beta
        )
    else:
        model_path = './trained_models/important_features_{}_{}_{}.pth'.format(
            model_name, dataset_params['sequence_length'], dataset_params['prediction_length']
        )
        
    print("Model path:", model_path)

    # Load pre-trained model or train a new one
    if os.path.exists(model_path):
        print(f"Loading the pre-trained {model_name} model...")
        model.load_state_dict(torch.load(model_path))
        model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    else:
        print(f"{model_name} model not found. Training a new model...")
        if 'LS_CNNLSTM' in model_name:
            train_for_long_term_forecast(
                model=model,
                model_name=model_name,
                train_data=train_data,
                train_targets=train_targets,
                eval_data=eval_data,
                eval_targets=eval_targets,
                model_path=model_path,
                num_epochs=num_epochs,
                batch_size=batch_size,
                learning_rate=learning_rate,
                patience=patience,
                oversample_eval=True, 
                alpha=mse_alpha, 
                beta=mse_beta
            )
        else:
            train_for_short_term_forecast(
                model=model,
                model_name=model_name,
                train_sequences=train_data,
                train_targets=train_targets,
                eval_sequences=eval_data,
                eval_targets=eval_targets,
                model_path=model_path,
                num_epochs=num_epochs,
                batch_size=batch_size,
                learning_rate=learning_rate,
                patience=patience
            )

    return model

In [22]:
top_5_resutls = {}


for key, important_features in important_features_dict.items():
    important_features_for_long = []
    important_features_for_short = []

    for feature_long in important_features['Important Long-term Features']:
        important_features_for_long.append(feature_long)
    
    for feature_short in important_features['Important Short-term Features']:
        important_features_for_short.append(feature_short)

    
    train_data, train_targets, eval_data, eval_targets, selected_features = load_dataset_new_features(params=dataset_params,
                                                                                                      is_long_term_forecast=True)

    
    # print(important_features)
    model_name = key.split('_long')[0]
    hidden_size = 512 if '512' in key else 256
    mse_decay = True if '0.3' in key else False

    model_params = {
        'model_name' : model_name, 
        'hidden_size' : hidden_size,
        'num_layers' : 3,  # 3
        'dropout' : 0.3,
        'num_epochs' : 200,
        'batch_size' : 16,
        'learning_rate' : 0.001,
        'patience' : 12,
        'mse_decay' : mse_decay
    }


    # build model 
    model = load_trained_model(model_params)
    
    print("Evaluating the model...")
    if is_long_term_forecast:
        results = evaluate_for_long_term_forecast(
            model=model,
            eval_data=eval_data,
            eval_targets=eval_targets,
            model_name=model_params['model_name'],
            batch_size=model_params['batch_size'], 
            oversample_eval=True 
        )
    else:
        results = evaluate_for_short_term_forecast(
            model=model,
            eval_sequences=eval_short,
            eval_targets=eval_targets_short,
            model_name=model_params['model_name'],
            batch_size=model_params['batch_size']
        )

    key = '{}_{}_{}'.format(model_name, hidden_size, mse_decay)
    top_5_resutls[key] = results

Model path: ./trained_models/important_features_11_11_LS_CNNLSTM_long_2160_short_720_256_256_3_alpha_1.0_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM model...
Evaluating the model...


/tmp/ipykernel_20448/4037395042.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


R² Score: 0.8371
Adjusted R²: 0.8368
SMAPE: 33.34
MASE: 0.5897
Model path: ./trained_models/important_features_11_11_LS_CNNLSTM_long_2160_short_720_256_256_3_alpha_0.3_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM model...
Evaluating the model...


/tmp/ipykernel_20448/4037395042.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


R² Score: 0.8720
Adjusted R²: 0.8717
SMAPE: 31.66
MASE: 0.5269
Model path: ./trained_models/important_features_11_11_LS_CNNLSTM_long_2160_short_720_256_512_3_alpha_0.3_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM model...
Evaluating the model...


...

Evaluating the model...
R² Score: 0.7161
Adjusted R²: 0.7156
SMAPE: 40.73
MASE: 0.7783
Model path: ./trained_models/important_features_11_11_LS_CNNLSTM_Att_long_2160_short_720_256_512_3_alpha_0.3_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM_Att model...


/tmp/ipykernel_20448/4037395042.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Evaluating the model...
R² Score: 0.7676
Adjusted R²: 0.7671
SMAPE: 37.42
MASE: 0.7044
Model path: ./trained_models/important_features_11_11_LS_CNNLSTM_Att_long_2160_short_720_256_512_3_alpha_1.0_beta_1.0.pth
Loading the pre-trained LS_CNNLSTM_Att model...


/tmp/ipykernel_20448/4037395042.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Evaluating the model...
R² Score: 0.4861
Adjusted R²: 0.4851
SMAPE: 133.01
MASE: 0.8530


In [23]:
top_5_resutls

{'LS_CNNLSTM_256_False': {'R2 Short': 0.8371367454528809,
  'Adjusted R2': 0.8368351975989812,
  'SMAPE Short': np.float32(33.336685),
  'MASE Short': np.float32(0.5897087)},
 'LS_CNNLSTM_256_True': {'R2 Short': 0.8719820380210876,
  'Adjusted R2': 0.8717450076252338,
  'SMAPE Short': np.float32(31.65939),
  'MASE Short': np.float32(0.5268928)},
 'LS_CNNLSTM_512_True': {'R2 Short': 0.9033562541007996,
  'Adjusted R2': 0.9031773143255275,
  'SMAPE Short': np.float32(28.621027),
  'MASE Short': np.float32(0.4620556)},
 'LS_CNNLSTM_512_False': {'R2 Short': 0.8045212030410767,
  'Adjusted R2': 0.8041592662010585,
  'SMAPE Short': np.float32(93.898224),
  'MASE Short': np.float32(0.66205746)},
 'LS_CNNLSTM_Att_256_False': {'R2 Short': 0.7808800935745239,
  'Adjusted R2': 0.7804743842712618,
  'SMAPE Short': np.float32(36.70067),
  'MASE Short': np.float32(0.68280476)},
 'LS_CNNLSTM_Att_256_True': {'R2 Short': 0.7161310315132141,
  'Adjusted R2': 0.7156054367222101,
  'SMAPE Short': np.float

In [36]:
original_results.keys() / top_5_resutls

dict_keys(['LS_CNNLSTM_256_True', 'LS_CNNLSTM_256_False', 'LS_CNNLSTM_512_True', 'LS_CNNLSTM_512_False', 'LS_CNNLSTM_Att_256_True', 'LS_CNNLSTM_Att_256_False', 'LS_CNNLSTM_Att_512_True', 'LS_CNNLSTM_Att_512_False'])

In [82]:
first_row = 'Model (w/ Params) \t\t\t\t R2(↑) \t\t\t Adj_R2(↑) \t\t SMAPE(↓) \t\t MASE(↓)'
print(first_row)

for model_type, performance in original_results.items():
    print_str = model_type + '\t\t\t\t'
    for eval_metric, measurement in performance.items():
        print_str += '{: .3f}'.format(measurement) + '\t\t\t'
    print_str += '\n'

    print_str += model_type + '(top 5)' + '\t\t\t'
    top_5_performance = top_5_resutls[model_type]
    for eval_metric, measurement in top_5_performance.items():
        print_str += '{: .3f}'.format(measurement) + '\t\t\t'
    print_str += '\n'

    print(print_str)

Model (w/ Params) 				 R2(↑) 			 Adj_R2(↑) 		 SMAPE(↓) 		 MASE(↓)
LS_CNNLSTM_256_True				 0.860			 0.859			 33.178			 0.553			
LS_CNNLSTM_256_True(top 5)			 0.872			 0.872			 31.659			 0.527			

LS_CNNLSTM_256_False				 0.849			 0.848			 33.692			 0.572			
LS_CNNLSTM_256_False(top 5)			 0.837			 0.837			 33.337			 0.590			

LS_CNNLSTM_512_True				 0.833			 0.833			 33.992			 0.596			
LS_CNNLSTM_512_True(top 5)			 0.903			 0.903			 28.621			 0.462			

LS_CNNLSTM_512_False				 0.835			 0.835			 34.892			 0.596			
LS_CNNLSTM_512_False(top 5)			 0.805			 0.804			 93.898			 0.662			

LS_CNNLSTM_Att_256_True				 0.726			 0.726			 39.989			 0.769			
LS_CNNLSTM_Att_256_True(top 5)			 0.716			 0.716			 40.731			 0.778			

LS_CNNLSTM_Att_256_False				 0.709			 0.708			 41.084			 0.792			
LS_CNNLSTM_Att_256_False(top 5)			 0.781			 0.780			 36.701			 0.683			

LS_CNNLSTM_Att_512_True				 0.778			 0.778			 38.919			 0.697			
LS_CNNLSTM_Att_512_True(top 5)			 0.768			 0.767			 37.424			 0.704			

L

{'LS_CNNLSTM_256_False': {'R2 Short': 0.8371367454528809,
  'Adjusted R2': 0.8368351975989812,
  'SMAPE Short': np.float32(33.336685),
  'MASE Short': np.float32(0.5897087)},
 'LS_CNNLSTM_256_True': {'R2 Short': 0.8719820380210876,
  'Adjusted R2': 0.8717450076252338,
  'SMAPE Short': np.float32(31.65939),
  'MASE Short': np.float32(0.5268928)},
 'LS_CNNLSTM_512_True': {'R2 Short': 0.9033562541007996,
  'Adjusted R2': 0.9031773143255275,
  'SMAPE Short': np.float32(28.621027),
  'MASE Short': np.float32(0.4620556)},
 'LS_CNNLSTM_512_False': {'R2 Short': 0.8045212030410767,
  'Adjusted R2': 0.8041592662010585,
  'SMAPE Short': np.float32(93.898224),
  'MASE Short': np.float32(0.66205746)},
 'LS_CNNLSTM_Att_256_False': {'R2 Short': 0.7808800935745239,
  'Adjusted R2': 0.7804743842712618,
  'SMAPE Short': np.float32(36.70067),
  'MASE Short': np.float32(0.68280476)},
 'LS_CNNLSTM_Att_256_True': {'R2 Short': 0.7161310315132141,
  'Adjusted R2': 0.7156054367222101,
  'SMAPE Short': np.float